In [52]:
from IPython.display import Image
import ee
ee.Initialize()
import sys 
sys.path.append('/rheil/notebooks/pygee')
import pandas as pd
import numpy as np
import simulation
import importlib
import gee_random
import gee_fc_tools as fct
importlib.reload(gee_random)
importlib.reload(simulation)
import matplotlib.pyplot as plt
%matplotlib inline  


In [53]:
asset_dir = 'users/rheilmayr/chile_sim/input/'
chile_comunas = ee.FeatureCollection('ft:1MIVtGXFEu3ONr87OPP7JV1_vCs9W3sJEuZDuzkzb')
# countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw');
# chile = countries.filterMetadata('Country', 'equals', 'Chile');
pixelAreaHa = ee.Image.pixelArea().divide(1e7)


Trying to dissolve to regions

In [54]:
chile_comunas = chile_comunas.filter(ee.Filter.inList('id_1', [15, 13, 8, 12, 6, 3, 10, 9]))
reducerDict = chile_comunas.reduceColumns(ee.Reducer.toList().group(), ['id_1', 'geometry']);
groupList = ee.List(reducerDict.get('groups'))
chile_regions = groupList.map(fct.dissolve_groups)

Code to clean up csv before moving to kml. Move elsewhere

In [55]:
def parse_key(key, text):
    text = text[text.find(key + '</td>\n\n<td>') + len(key + '</td>\n\n<td>'):]
    out_text = text[:text.find('<')]
    return out_text

comunas_csv = '/rheil/notebooks/data/CHL_adm3.csv'
comunas_df = pd.read_csv(comunas_csv)
for i in range(4):
    comunas_df['id_' + str(i)] = comunas_df['description'].apply(lambda x: parse_key('ID_' + str(i), x))
    comunas_df['name_' + str(i)] = comunas_df['description'].apply(lambda x: parse_key('NAME_' + str(i), x))
comunas_df = comunas_df.drop('name', axis = 1)
comunas_df.to_csv('/rheil/notebooks/data/CHL_adm3_parsed.csv')

In [56]:
clas_1986 = ee.Image(asset_dir + 'css_1986')
clas_2001 = ee.Image(asset_dir + 'css_2001')
clas_2011 = ee.Image(asset_dir + 'css_2011')

In [57]:
mask = clas_1986.gte(1).And(clas_1986.lte(19))\
    .And(clas_2001.gte(1).And(clas_2001.lte(19)))\
    .And(clas_2011.gte(1).And(clas_2011.lte(19)))
clas_1986 = clas_1986.updateMask(mask)
clas_2001 = clas_2001.updateMask(mask)
clas_2011 = clas_2011.updateMask(mask)

In [58]:
def clas_area_band(lu_img, clas_val, band_name):
    clas_img = lu_img.eq(clas_val)
    clas_area_img = clas_img.multiply(ee.Image.pixelArea().divide(1e7))
    clas_area_img = clas_area_img.select(['b1'], [band_name])
    return clas_area_img

c1986_forarea_img = clas_area_band(clas_1986, 1, 'c_for_1986')
c1986_plantarea_img = clas_area_band(clas_1986, 3, 'c_plant_1986')
c2001_forarea_img = clas_area_band(clas_2001, 1, 'c_for_2001')
c2001_plantarea_img = clas_area_band(clas_2001, 3, 'c_plant_2001')
c2011_forarea_img = clas_area_band(clas_2011, 1, 'c_for_2011')
c2011_plantarea_img = clas_area_band(clas_2011, 3, 'c_plant_2011')

In [59]:
hansen_img = ee.Image('UMD/hansen/global_forest_change_2015').updateMask(mask)
fc_img = hansen_img.select('treecover2000')
for2000_img = fc_img.gte(25)
lossyear_img = hansen_img.select('lossyear')
loss_img = hansen_img.select('loss')
gain_img = hansen_img.select('gain')
for2000area_img = for2000_img.multiply(pixelAreaHa)
lossarea_img = (for2000_img.eq(1).And(lossyear_img.lte(12)).And(loss_img.eq(1))).multiply(pixelAreaHa)
gainarea_img = gain_img.multiply(pixelAreaHa)
for2000area_img = for2000area_img.select(['treecover2000'], ['hansen_for_2000'])
lossarea_img = lossarea_img.select(['treecover2000'], ['hansen_loss'])
gainarea_img = gainarea_img.select(['gain'], ['hansen_gain'])


In [60]:
cat1_img = ee.Image(asset_dir + 'cadaster_p1').updateMask(mask)
cat1_for_img = cat1_img.eq(1).Or(cat1_img.eq(2)).Or(cat1_img.eq(7))
cat1_forarea_img = cat1_for_img.multiply(pixelAreaHa)
cat1_forarea_img = cat1_forarea_img.select(['b1'], ['cat_for_p1'])
cat2_img = ee.Image(asset_dir + 'cadaster_p2').updateMask(mask)
cat2_for_img = cat2_img.eq(1).Or(cat2_img.eq(2)).Or(cat2_img.eq(7))
cat2_forarea_img = cat2_for_img.multiply(pixelAreaHa)
cat2_forarea_img = cat2_forarea_img.select(['b1'], ['cat_for_p2'])
cat1_plantarea_img = clas_area_band(cat1_img, 4, 'cat_plant_p1')
cat2_plantarea_img = clas_area_band(cat2_img, 4, 'cat_plant_p2')


In [61]:
bands = [cat1_forarea_img, cat1_plantarea_img, cat2_forarea_img, cat2_plantarea_img, gainarea_img, 
         for2000area_img, lossarea_img, c1986_forarea_img, c1986_plantarea_img, c2001_forarea_img, 
         c2001_plantarea_img, c2011_forarea_img, c2011_plantarea_img]
area_img = ee.Image.cat(bands)

In [62]:
chile = area_img.reduceRegions(collection = chile_comunas, scale = 30, reducer = ee.Reducer.sum())

In [63]:
task = ee.batch.Export.table.toDrive(collection = chile, description = 'lu_compare_maskor', folder = 'chile', fileFormat = 'csv')
task.start()

In [65]:
task = ee.batch.Export.image.toDrive(image = clas_2011, description = 'clas_2011', 
                                     folder = 'chile', fileNamePrevix = 'clas_2011', 
                                     scale = 30, maxPixels = 1e10)
task.start()